In [36]:
import pandas as pd
from collections import defaultdict
import numpy as np

In [2]:
df = pd.read_csv('../data/filtered_data.csv')

/Users/vincenthuang/.virtualenvs/comp598-hw4/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (21,32,33,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df_without_na = df[df['Incident Zip'].notnull()]
df_without_na = df_without_na[df_without_na['Created Date'].notnull()]
df_without_na = df_without_na[df_without_na['Closed Date'].notnull()]
df_without_na = df_without_na.drop('Unnamed: 0', 1)
df_without_na['Created Date'] = pd.to_datetime(df_without_na['Created Date'])
df_without_na['Closed Date'] = pd.to_datetime(df_without_na['Closed Date'])
df_with_valid_timeline = df_without_na[df_without_na['Created Date'] < df_without_na['Closed Date']]

/var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/ipykernel_89216/300208126.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_without_na = df_without_na.drop('Unnamed: 0', 1)


In [47]:
df_with_valid_timeline['Time Difference'] = round((df_with_valid_timeline['Closed Date'] - df_with_valid_timeline['Created Date']) / np.timedelta64(1, 'h'))
df_with_valid_timeline['Months'] = pd.DatetimeIndex(df_with_valid_timeline['Closed Date']).month

/var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/ipykernel_89216/2258941495.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_valid_timeline['Time Difference'] = round((df_with_valid_timeline['Closed Date'] - df_with_valid_timeline['Created Date']) / np.timedelta64(1, 'h'))
/var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/ipykernel_89216/2258941495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_valid_timeline['Months'] = pd.DatetimeIndex(df_with_valid_timeline['Closed Date']).month


In [58]:
# for each zip code compute its monthly average incident create-to-closed time (in hours)
zip_codes = df_with_valid_timeline['Incident Zip'].unique().tolist()
zip_codes.append('all')
result = defaultdict(lambda : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # average response time of 12 months initialized
for zip_code in zip_codes:
    if zip_code == 'all':
        rows_at_zip_code = df_with_valid_timeline
    else:
        rows_at_zip_code = df_with_valid_timeline[df_with_valid_timeline['Incident Zip'] == zip_code]
    # get average for each month
    for i in range(0, 12):
        # rows that belongs to month i + 1
        rows_at_month = rows_at_zip_code[rows_at_zip_code['Months'] == i + 1]
        if len(rows_at_month) > 0:
            result[zip_code][i] = sum(rows_at_month['Time Difference']) / len(rows_at_month)
        

In [78]:
average_resp_time = pd.DataFrame.from_dict(result, orient='index', columns=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec'])

In [79]:
average_resp_time.

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec
11234.0,108.613198,228.843641,119.893241,128.329039,80.878179,75.598843,119.511637,151.643465,305.609665,0,0,0
11210.0,61.280528,99.499466,85.711042,86.534207,81.461456,134.859674,188.014749,178.846497,270.616883,0,0,0
11354.0,45.770328,72.393759,75.437026,100.060932,103.398085,110.921519,251.362319,212.726843,324.823105,0,0,0
10451.0,41.960638,72.034330,80.039572,88.965583,90.814888,183.291162,82.732910,101.448894,109.538028,0,0,0
10031.0,46.021909,79.611111,74.621244,36.440998,69.808519,109.117119,97.598122,109.941619,103.088075,0,0,0


In [80]:
average_resp_time.to_csv('../data/average.csv')

In [87]:
df = pd.read_csv('../data/average.csv')
new_df = df.rename(columns={'Unnamed: 0': 'Zip Codes'})
new_df.to_csv('../data/average.csv', index=False)

,Zip Codes,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec
0,11234.0,108.613198,228.843641,119.893241,128.329039,80.878179,75.598843,119.511637,151.643465,305.609665,0,0,0
1,11210.0,61.280528,99.499466,85.711042,86.534207,81.461456,134.859674,188.014749,178.846497,270.616883,0,0,0
2,11354.0,45.770328,72.393759,75.437026,100.060932,103.398085,110.921519,251.362319,212.726843,324.823105,0,0,0
3,10451.0,41.960638,72.034330,80.039572,88.965583,90.814888,183.291162,82.732910,101.448894,109.538028,0,0,0
4,10031.0,46.021909,79.611111,74.621244,36.440998,69.808519,109.117119,97.598122,109.941619,103.088075,0,0,0
